# Scrape the odds
Website: https://www.unibet.eu/betting#filter/football/

From Unibet using Selenium and Firefox.

In [1]:
import numpy as np
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from time import sleep

In [15]:
# quit the browser
browser.quit()

## Initialize the browser

In [53]:
# start the browser
browser = webdriver.Firefox()

# have a generic wait object for the driver
wait = WebDriverWait(browser, 10)

In [54]:
# go to Unibet
url_0 = 'https://www.unibet.eu/betting#filter/football/'
browser.get(url_0)

# wait until the accept cookies button is clickable
try:
    element = wait.until(
        EC.element_to_be_clickable(
            (By.ID, "CybotCookiebotDialogBodyButtonAccept")
        )
    )
    # click on it
    element.click()
except TimeoutException:
    # not found, but probably already clicked before so continue
    import warnings
    warnings.warn('Element CybotCookiebotDialogBodyButtonAccept not found. Maybe something went wrong or maybe it''s already clicked.')
    pass
finally:
    # wait until the cookie bar is really gone
    wait.until(
        EC.invisibility_of_element_located(
            (By.ID, "CybotCookiebotDialog")
        )
    )

## Visit some leagues for scraping

In [19]:
# the urls we'll visit
urls = [url_0 + u for u in ['netherlands/eredivisie',
                            'spain/la_liga',
                            'germany/bundesliga',
                            'england/premier_league',
                            'france/ligue_1',
                            'italy/serie_a']]
urls

['https://www.unibet.eu/betting#filter/football/netherlands/eredivisie',
 'https://www.unibet.eu/betting#filter/football/spain/la_liga',
 'https://www.unibet.eu/betting#filter/football/germany/bundesliga',
 'https://www.unibet.eu/betting#filter/football/england/premier_league',
 'https://www.unibet.eu/betting#filter/football/france/ligue_1',
 'https://www.unibet.eu/betting#filter/football/italy/serie_a']

In [82]:
# fcn: extract match info from a <li.KambiBC-event-item>
def extractMatchInfo(match):
    teams = match.select('.KambiBC-event-participants__name')
    odds = match.select('span.KambiBC-mod-outcome__odds')
    info = {'home_team': teams[0].text,
            'away_team': teams[1].text,
            'odd_home_win': odds[0].text,
            'odd_tie': odds[1].text,
            'odd_away_win': odds[2].text
           }
    return info

### Crawl urls and get the soups

In [61]:
# wait for page to be done loading
def pageReady(interval=1):
    # in seconds
    waited = 0
    while True:
        soupCheck1 = BeautifulSoup(browser.page_source, 'html5lib')
        waited += interval
        sleep(interval)
        soupCheck2 = BeautifulSoup(browser.page_source, 'html5lib')
        if soupCheck1 == soupCheck2:
            break
    return waited

In [55]:
# create empty dict to append the soup of each league
soups = dict()

# loop over all urls
for url in urls:
    
    # debug
    url = urls[0]
    
    # go to url
    print('Going to:', url)
    browser.get(url)

    # click on all the dropdown elements
    while True:
        try:
            print('.')
            # wait and find a dropdown element, ignore the already expanded ones
            dropdown = browser.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR,
                '.KambiBC-collapsible-container:not(.KambiBC-expanded)')
            ))
            print(dropdown)
            # click on it
            dropdown.click()
        except TimeoutException:
            # not found, break the loop and continue with script
            break

    print('hi')
    
    # wait until all (how all?) green buttons are present
    wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, 'button.KambiBC-mod-outcome')
    ))

    # now get the beautiful source code
    soup = BeautifulSoup(browser.page_source, 'html5lib')
    
    # select and store only the list of matches
    league = re.split('/', url)[-1]
    soups[league] = soup.select_one('.KambiBC-event-groups-list')
    
    # debug
    break

Going to: https://www.unibet.eu/betting#filter/football/netherlands/eredivisie
.
hi


TimeoutException: Message: 


In [60]:
# create empty dict to append the soup of each league
soups = dict()

# loop over all urls
for url in urls:
    
    # debug
    url = urls[1]
    
    # go to url
    print('Going to:', url)
    browser.get(url)
    
    # wait until page is ready
    print('Page ready in', pageReady())

    # click on all the dropdown elements, ignore the already expanded ones
    dropdowns = browser.find_elements(By.CSS_SELECTOR,
        '.KambiBC-collapsible-container:not(.KambiBC-expanded)')
    for dropdown in dropdowns:
        dropdown.click()

    # wait again
    print('Page ready in', pageReady())

    # now get the beautiful source code
    soup = BeautifulSoup(browser.page_source, 'html5lib')
    
    # select and store only the list of matches
    league = re.split('/', url)[-1]
    soups[league] = soup.select_one('.KambiBC-event-groups-list')
    
    # debug
    break

Going to: https://www.unibet.eu/betting#filter/football/spain/la_liga
Page ready in 2
2


In [7]:
    # find all green buttons (with odds)
    greenButtons = soup.select('.KambiBC-event-groups button.KambiBC-mod-outcome')

    # find the <li> parent which represents one match and combine them as a set
    matches = list(set(o.find_parent('li', class_='KambiBC-event-item') for o in greenButtons))

    # extract info of all matches and store in a data frame
    df_0 = pd.DataFrame([extractMatchInfo(m) for m in matches])

    # append to main data frame
    df = pd.concat([df, df_0])
    
    break

In [64]:
# find all green buttons (with odds)
greenButtons = soup.select('.KambiBC-event-groups button.KambiBC-mod-outcome')

# find the <li> parent which represents one match and combine them as a set
matches = list(set(o.find_parent('li', class_='KambiBC-event-item') for o in greenButtons)

# extract match info (team names and odds) from a <li.KambiBC-event-item>
def extractMatchInfo(match):
    teams = match.select('.KambiBC-event-participants__name')
    odds = match.select('span.KambiBC-mod-outcome__odds')
    info = {'home_team': teams[0].text,
            'away_team': teams[1].text,
            'odd_home_win': odds[0].text,
            'odd_tie': odds[1].text,
            'odd_away_win': odds[2].text
           }
    return info

# extract info of all matches and store in a data frame
pd.DataFrame([extractMatchInfo(m) for m in matches])

### Data transformation

In [ ]:
# Change team names to lowercase
df["home_team"] = df["home_team"].str.lower()
df["away_team"] = df["away_team"].str.lower()

# Remove all accents
df["home_team"] = df["home_team"].str.normalize('NFKD').str.encode(encoding='ascii',errors='ignore').str.decode('utf-8')
df["away_team"] = df["away_team"].str.normalize('NFKD').str.encode(encoding='ascii',errors='ignore').str.decode('utf-8')

# Convert to numbers
df["odd_home_win"] = pd.to_numeric(df["odd_home_win"])
df["odd_away_win"] = pd.to_numeric(df["odd_away_win"])
df["odd_tie"] = pd.to_numeric(df["odd_tie"])

# Show
df

In [ ]:
# append to csv
df.to_csv('./data/This_month_odds.csv', index=False, mode='a')